In [ ]:
@log_errors(ect_logger)
async def get_enhanced_ect(
    self,
    ticker: Optional[str] = None,
    bbg_cid: Optional[str] = None,
    refinitiv_id: Optional[str] = None,
    year: Optional[int] = None,
    quarter: Optional[str] = None
) -> Dict[str, Any]:
    """
    Get enhanced ECT analysis combining both ECT summary and supply chain analysis
    
    Args:
        ticker: Optional company ticker
        bbg_cid: Optional Bloomberg company ID
        refinitiv_id: Optional Refinitiv company ID
        year: Optional year filter
        quarter: Optional quarter filter
        
    Note: Exactly one of ticker, bbg_cid, or refinitiv_id must be provided
    """
    try:
        # Validate input parameters
        provided_ids = [
            id_type for id_type, value in [
                ("ticker", ticker),
                ("bbg_cid", bbg_cid),
                ("refinitiv_id", refinitiv_id)
            ] if value is not None
        ]
        
        if len(provided_ids) == 0:
            raise ValueError("At least one identifier (ticker, bbg_cid, or refinitiv_id) must be provided")
        if len(provided_ids) > 1:
            raise ValueError(f"Multiple identifiers provided: {provided_ids}. Please provide only one.")

        # Initialize RelationDB
        rel_db = RelationDB(year=year, quarter=quarter, use_cache=True)
        
        # Get company info based on provided identifier
        if ticker:
            ect_logger.info(f"Starting enhanced ECT analysis for ticker: {ticker}")
            bbg_cid = rel_db.ticker_to_bbg_cid(ticker)
            if not bbg_cid:
                raise ValueError(f"Could not find BBG CID for ticker {ticker}")
                
        elif bbg_cid:
            ect_logger.info(f"Starting enhanced ECT analysis for BBG CID: {bbg_cid}")
            ticker = rel_db.bbg_cid_to_ticker(bbg_cid)
            if not ticker:
                raise ValueError(f"Could not find ticker for BBG CID {bbg_cid}")
                
        elif refinitiv_id:
            ect_logger.info(f"Starting enhanced ECT analysis for Refinitiv ID: {refinitiv_id}")
            company_info = rel_db.get_company_info_by_refinitiv_id(refinitiv_id)
            if not company_info:
                raise ValueError(f"Could not find company info for Refinitiv ID {refinitiv_id}")
            ticker = company_info.ticker
            bbg_cid = company_info.bbg_cid

        # Get complete company info
        company_info = rel_db.get_company_info(bbg_cid)
        if not company_info or not company_info.refinitiv_id:
            raise ValueError(f"Could not find complete company info for BBG CID {bbg_cid}")

        # Get ECT summary and save responses for confidence scores
        content = await self.get_latest_earning([company_info.refinitiv_id])
        if not content:
            raise ContentFetchError("Empty content received")
            
        prompts = self.generate_ect_prompts(content[0].transcript, bbg_cid)
        responses = await self.execute_prompts(prompts)
        
        analysis_sections = {}
        confidence_scores = {}
        for response in responses:
            if response["confidence"] > self.config.llm.confidence_threshold:
                analysis_dict = response["result"]
                analysis_sections.update(analysis_dict)
                # Store confidence scores by section
                confidence_scores[response.get("heading", f"section_{response['bucket']}")] = response["confidence"]
                
        ect_summary = ECTResponse(
            ect_metadata=content[0].metadata,
            ect_summary=analysis_sections
        )
        
        # Get supply chain analysis
        supply_chain = await self.get_supply_chain_analysis(
            ticker=ticker,
            year=year,
            quarter=quarter
        )

        # Process supply chain confidence scores
        supply_chain_confidence = {}
        if isinstance(supply_chain, list):
            for analysis in supply_chain:
                if "confidence" in analysis:
                    supply_chain_confidence[analysis["type"]] = analysis["confidence"]

        # Combine results
        result = {
            "company_info": {
                "name": company_info.name,
                "ticker": ticker,
                "bbg_cid": bbg_cid,
                "refinitiv_id": company_info.refinitiv_id
            },
            "analysis_date": datetime.now().isoformat(),
            "analysis_period": {
                "year": year,
                "quarter": quarter
            },
            "ect_summary": {
                "metadata": ect_summary.ect_metadata,
                "analysis": ect_summary.ect_summary,
                "confidence_scores": confidence_scores
            },
            "supply_chain_analysis": {
                "analysis": supply_chain,
                "confidence_scores": supply_chain_confidence
            }
        }

        ect_logger.info(
            f"Enhanced ECT analysis completed for {company_info.name}"
        )
        return result

    except ECTError as e:
        ect_logger.error(f"ECT error in enhanced analysis: {e}")
        raise
    except Exception as e:
        ect_logger.error(f"Unexpected error in enhanced analysis: {e}")
        raise ECTError(f"Failed to generate enhanced ECT analysis: {str(e)}")